In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from utils import crop_image, circle_crop, temp_augmentate
import cv2
import matplotlib.pyplot as plt
import shutil

In [2]:
def selectRandomImage():
    import random # Rastgelelik için random kütüphanesini kullanıyoruz
    mylist = list() # Fotoğrafları eklemek için bir liste oluşturuyoruz
    for r, _, f in os.walk(".\\images"): # ./animals/animals klasörü altındaki tüm dosyaları geziyoruz
        if len(f) > 0: # Bulunduğumuz klasördeki  dosya sayısı 0 dan büyükse işlem yapıyoruz
            file = random.choice(f) # Klasör içinden (örneğin kedi klasörü) bir fotoğraf seçiyoruz.
            # Daha sonra bu fotoğrafı ve fotoğraftaki hayvanın adını listemize ekliyoruz.
            mylist.append((f"{r}\\{file}".replace("\\", "/"), f"{r}\\{file}".replace("\\", "/").split("/")[-2]))
    return random.choice(mylist) # Listeden rastgele bir eleman döndürüyoruz.
selectRandomImage()

('./images/4/43839_right.jpeg', '4')

In [3]:
def preprocess(imgPath : str = None, imgClass = None):
    if imgPath == None:
        imgObj = selectRandomImage()
    else:
        imgObj = (imgPath, imgClass)
    #print(imgObj[1])
    img = cv2.imread(imgObj[0])
    img_cropped, flag = crop_image(img)
    height, width, _= img_cropped.shape
    ratio = height/width
    new_sz = 1024
    if width > new_sz:
        new_image = cv2.resize(img_cropped,(new_sz,math.ceil(new_sz*ratio)))  
    else:
        new_image = img_cropped

    circle_cropped = circle_crop(new_image, 30)
    if imgPath == None:
        return (circle_cropped, imgObj[1])
    else:
        return circle_cropped

# preprocessed_img = preprocess()
# plt.imshow(preprocessed_img)
# _ = cv2.imwrite("./deneme.jpeg", preprocessed_img)

In [4]:
def augmentate():
    imgObject = selectRandomImage()
    shutil.copy(imgObject[0], "./temp/img.jpeg")
    temp_augmentate()
    objs = []
    for fileName in os.listdir("./temp/output"):
        objs.append(preprocess("./temp/output/" + str(fileName), imgObject[1]))
    return (objs, imgObject[1])
        
#augmentateObj = augmentate()

In [5]:
model = keras.models.load_model("079acc.mdl_wts.hdf5")

In [6]:
def predictAugmented(augmentateObj):
    ## Bir fotoğrafı çoğaltıp tahmin ettir.
    
    class_names = ["0", "1", "2", "3", "4"]
    predictions_dict = {"0" : 0, "1" : 0, "2" : 0, "3" : 0, "4" : 0}
    for i in augmentateObj[0]:
        img_array, cls = i, augmentateObj[1]
        img_array = tf.expand_dims(img_array, 0) # Veri listesindeki verileri paket halinde hazırlıyoruz.

        predictions = model.predict(img_array) # Bu paketi modelimize veriyoruz ve tahminini yapıyor.
        score = tf.nn.softmax(predictions[0]) # Tahmin sonucunu alıyoruz. Softmax aldığımız değerlerin 0 ile 1 aralığında olmasını sağlıyor
        predictions_dict[class_names[np.argmax(score)]] += 1

    print(predictions_dict)
    print(str(augmentateObj[1]))
    print(str(max(predictions_dict, key=predictions_dict.get)))
    return str(max(predictions_dict, key=predictions_dict.get)) == str(augmentateObj[1])
#predictAugmented(augmentateObj)

In [7]:
def predictRandom():
    ## Bir fotoğrafı çoğaltıp tahmin ettir.
    
    class_names = ["0", "1", "2", "3", "4"]
    img_array, cls = preprocess()
    img_array = tf.expand_dims(img_array, 0) # Veri listesindeki verileri paket halinde hazırlıyoruz.

    predictions = model.predict(img_array) # Bu paketi modelimize veriyoruz ve tahminini yapıyor.
    score = tf.nn.softmax(predictions[0]) # Tahmin sonucunu alıyoruz. Softmax aldığımız değerlerin 0 ile 1 aralığında olmasını sağlıyor


    # Modelin tahminini doğruluk payı ile beraber ekrana bastırıyoruz.
    #print(
    #    "This image most likely belongs to {} with a %{:.2f} confidence."
    #    .format(class_names[np.argmax(score)], 100 * np.max(score))
    #)
    return cls == class_names[np.argmax(score)]
#predictRandom()

In [8]:
def clearOutput():
    for f in os.listdir("./temp/output"):
        if not f.endswith(".jpeg"):
            continue
        os.remove(os.path.join("./temp/output", f))

In [9]:
def predictMultiple():
    t_count = 0
    f_count = 0
    for i in range(10):
        augmentateObj = augmentate()
        if predictAugmented(augmentateObj):
            print(True)
            t_count += 1
        else:
            print(False)
            f_count += 1
        clearOutput()

    print(t_count, f_count, "%"+str((t_count / (t_count + f_count)) * 100))

In [10]:
def predictMultipleRandom():
    t_count = 0
    f_count = 0
    for i in range(1000):
        if predictRandom():
            t_count += 1
        else:
            f_count += 1

    print(t_count, f_count, "%"+str((t_count / (t_count + f_count)) * 100))

In [11]:
predictMultiple()

Processing <PIL.Image.Image image mode=RGB size=1024x683 at 0x23E3086C040>: 100%|█| 1/1 [00:00<00:00, 111.10 Samples/s]
Executing Pipeline:   0%|                                                                  | 0/9 [00:00<?, ? Samples/s]

Initialised with 1 image(s) found.
Output directory set to ./temp\output.

Processing <PIL.Image.Image image mode=RGB size=1024x683 at 0x23E095DBCD0>: 100%|██| 9/9 [00:00<00:00, 40.54 Samples/s]
Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x768 at 0x23E1C987520>: 100%|█| 1/1 [00:00<00:00,
Executing Pipeline:   0%|                                                                  | 0/9 [00:00<?, ? Samples/s]

{'0': 9, '1': 0, '2': 0, '3': 3, '4': 8}
0
0
True
Initialised with 1 image(s) found.
Output directory set to ./temp\output.

Processing <PIL.Image.Image image mode=RGB size=1024x768 at 0x23E0FCC11F0>: 100%|██| 9/9 [00:00<00:00, 37.04 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=1024x768 at 0x23E4CF706A0>: 100%|█| 1/1 [00:00<00:00, 111.09 Samples/s]
Executing Pipeline:   0%|                                                                  | 0/9 [00:00<?, ? Samples/s]

...

{'0': 0, '1': 2, '2': 1, '3': 1, '4': 6}
3
4
False
Initialised with 1 image(s) found.
Output directory set to ./temp\output.

Processing <PIL.Image.Image image mode=RGB size=1024x683 at 0x23E1ADE8520>: 100%|██| 9/9 [00:00<00:00, 52.33 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=1024x683 at 0x23E36059C40>: 100%|█| 1/1 [00:00<00:00, 100.00 Samples/s]
Executing Pipeline:   0%|                                                                  | 0/9 [00:00<?, ? Samples/s]

{'0': 2, '1': 3, '2': 5, '3': 0, '4': 0}
1
2
False
Initialised with 1 image(s) found.
Output directory set to ./temp\output.

Processing <PIL.Image.Image image mode=RGB size=1024x683 at 0x23E3589F220>: 100%|██| 9/9 [00:00<00:00, 36.29 Samples/s]


{'0': 10, '1': 0, '2': 0, '3': 0, '4': 0}
1
0
False
3 7 %30.0
